## Todos
- Make a pull request/reopen that one issue
- PRF fit -- extract stellar/source flux? if so, you can multiply the 1d lightcurve by the stellar flux 

In [16]:
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np

from lightkurve import KeplerTargetPixelFile, LightCurve
from lightkurve.targetpixelfile import TargetPixelFileFactory
from lightkurve.prf import StarPrior, BackgroundPrior, FocusPrior, MotionPrior, TPFModel, PRFPhotometry
from oktopus import GaussianPrior, UniformPrior

In [17]:
lk

<module 'lightkurve' from '/Users/ssagear/github/lightkurve/src/lightkurve/__init__.py'>

In [23]:
tpf = KeplerTargetPixelFile("https://archive.stsci.edu/missions/kepler/target_pixel_files/0084/008462852/"
                            "kplr008462852-2013098041711_lpd-targ.fits.gz", quality_mask='hardest')

In [24]:
tpf.flux

<Quantity [[[            nan, -5.74946070e+00, -1.13220179e+00, ...,
              2.01544144e+02,  1.22487183e+03,             nan],
            [            nan,  3.17240925e+01,  3.02265759e+01, ...,
              7.99651718e+01,  6.66418701e+02,             nan],
            [ 9.51571045e+01,  5.42663460e+01,  2.01943283e+02, ...,
              1.39809616e+02,  1.98060379e+02,             nan],
            ...,
            [            nan,  8.34692383e+01,  6.13087273e+01, ...,
              5.06303406e+02,  4.22942047e+02,  1.39690872e+02],
            [            nan,  2.19871597e+01,  3.73812675e+01, ...,
              4.83090027e+02,  1.14936127e+02,             nan],
            [            nan, -2.38509321e+00,  1.21481033e+02, ...,
              2.51940125e+02,  5.43092308e+01,             nan]],

           [[            nan, -2.82080412e+00,  2.76082009e-01, ...,
              2.04003906e+02,  1.22939441e+03,             nan],
            [            nan,  3.47096252e+

In [5]:
oops = tpf.extract_prf_photometry()

Fitting cadences: 100%|██████████████████████████████████████████| 3540/3540 [01:19<00:00, 44.68it/s]


In [6]:
oops.results[0]

  <StarParameters(
    col=232.46966094957332
    row=131.78401736836238
    flux=241646.46629568693
    targetid=None)>
  <BackgroundParameters(
    flux=459.8533634264026
    fitted=True)>
  <FocusParameters(
    scale_col=[1]
    scale_row=[1]
    rotation_angle=[0.]
    fitted=False)>
  <MotionParameters(
    shift_col=-0.0883256271481514
    shift_row=-0.22839058935642242
    fitted=False)>
  residual_image:
    [          nan -135.24134612 -147.49296708 -156.02464029...
  predicted_image:
    [445.36797008 447.74369597 464.58992143 477.12763711...

In [7]:
import batman
import matplotlib.pyplot as plt

params = batman.TransitParams()
params.t0 = 0.
params.per = 1.
params.rp = 1.5
params.a = 15.
params.inc = 87.
params.ecc = 0.
params.w = 90.
params.limb_dark = "nonlinear"
params.u = [0.5, 0.1, 0.1, -0.1]

t = np.linspace(-0.025, 0.025, len(oops.results))
m = batman.TransitModel(params, t)

b_flux = m.light_curve(params)

In [6]:
for i in range(len(oops.results)):
    oops.results[i].stars[0].flux = oops.results[i].stars[0].flux*b_flux[i]

In [72]:
# from moviepy.editor import VideoClip
# from moviepy.video.io.bindings import mplfig_to_npimage
    
# # duration of the video
# duration = 20

# # matplot subplot
# fig, ax = plt.subplots()

# # method to get frames
# def make_frame(t):

#     # clear
#     ax.clear()
#     # plotting line
#     x = int(t * 177)
#     ax.imshow(oops.model.predict(params=oops.results[x]), origin='lower', vmin=0, vmax=50000)


#     # returning numpy image
#     return mplfig_to_npimage(fig)

# # creating animation
# animation = VideoClip(make_frame, duration = duration)
# animation.ipython_display(fps=177)


In [98]:
factory = TargetPixelFileFactory(n_cadences=1, n_rows=len(tpf.flux[0]), n_cols=len(tpf.flux[0]))


In [99]:
factory.add_cadence(
                frameno=0,
                raw_cnts=np.ones_like(tpf.flux[0]),
                flux=tpf.flux[0],
                flux_err=tpf.flux_err[0],
                flux_bkg=tpf.flux_bkg[0],
                flux_bkg_err=tpf.flux_bkg_err[0],
            )

In [100]:
# hdu_idx = tpf.get_header()
# factory.keywords = hdu_idx

In [101]:
tpfm = factory.get_tpf()

In [91]:
tpfm.quality_mask = np.array(np.ones_like(tpf.flux)).astype('int')

In [102]:
tpfm.flux

AttributeError: 'TargetPixelFile' object has no attribute 'quality_mask'